In [1]:
from torch import nn, optim
import torch
from collections import namedtuple
import random
import pandas
from itertools import count
import math
from torch.autograd import Variable


In [2]:
data = pandas.DataFrame.from_csv('final.csv')
data = data.values
prices = pandas.DataFrame.from_csv('xmrbtc.csv')


In [3]:
use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor
Tensor = FloatTensor

In [4]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))
class ReplayMemory():
    
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0
        
    def push(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity
        
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)
            

In [5]:
class TNetwork(nn.Module):
    def __init__(self, input_size, hidden_size=256, output_size=3):
        super(TNetwork, self).__init__()
        self.full1 =  nn.Linear(input_size + hidden_size, hidden_size)
        self.full2 =  nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.Linear(output_size, 3)
        self.hidden_size = hidden_size
        
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden),1)
        hidden = self.full1(combined)
        output = self.full2(combined)
        output = self.softmax(output)
        return output, hidden
    
    def init_hidden(self):
        return Variable(torch.zeros(1, self.hidden_size)).type(FloatTensor)


In [6]:
class DataIter:
    def __init__(self, data):
        self.data = data
        self.index = 0
        
    def __iter__(self):
        return self
    
    def next(self):
        if self.index >= len(self.data):
            self.index = 0
        d = self.data[self.index]
        self.index += 1
        return d
    
    def current(self):
        return self.data[self.index]
    
    def index(self):
        return self.index
    
    def has_next(self):
        return self.index < len(self.data)

it = DataIter(data)
def get_next_data():
    return torch.from_numpy(it.next()).unsqueeze(0).type(Tensor)

class MoneySimulator:
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.monero = 0.1
        self.btcMax = self.monero * prices['close'].iloc[0]
        self.btc = 0
        
    def step(self,action):
        if action == 1:
            return (0, False)
        
        if action == 0 and self.monero == 0:
            return  (0, False)
        
        if action == 2 and self.btc == 0:
            return  (0, False)
        
        if action == 0:
            self.monero = self.monero / 2
            xmr_p = self.monero
            self.btc = xmr_p * prices['close'].iloc[it.index + 1]
            
            current = self.monero / prices['close'].iloc[it.index + 1] + self.btc
            
            self.btcMax = self.btcMax if current < self.btcMax else current
            return self.calculate_res()
        
        if action == 2:
            self.btc = self.btc / 2
            btc = self.btc
            self.monero = btc / prices['close'].iloc[it.index + 1]
            
            current = self.monero / prices['close'].iloc[it.index + 1] + self.btc
            
            self.btcMax = self.btcMax if current < self.btcMax else current
            return self.calculate_res()
        
    def calculate_res(self):
        def calculate_at_moment(price):
            btc = self.monero / price
            return (btc + self.btc) - self.btcMax
        
        return (calculate_at_moment(prices['close'].iloc[it.index + 2]), False)
        
        
sim = MoneySimulator()

In [7]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200

model = TNetwork(len(data[0]))

if use_cuda:
    model.cuda()
    
optimizer = optim.RMSprop(model.parameters())
memory = ReplayMemory(10000)

steps_done = 0
    
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    hidden = model.init_hidden()
    if sample > eps_threshold:
        output, hidden = model(
            Variable(state, volatile=True).type(FloatTensor),hidden)
        return output.data.max(1)[1].view(1, 1)
    else:
        return LongTensor([[random.randrange(3)]])



In [8]:
def optimize_model():
    global last_sync
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    non_final_mask = ByteTensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)))
    print(f'not working ${batch.state}')

    # We don't want to backprop through the expected action values and volatile
    # will save us on temporarily changing the model parameters'
    # requires_grad to False!
    non_final_next_states = Variable(torch.cat([s for s in batch.next_state
                                                if s is not None]),
                                     volatile=True)
    state_batch = Variable(torch.cat(batch.state))
    action_batch = Variable(torch.cat(batch.action))
    reward_batch = Variable(torch.cat(batch.reward))
    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken
    state_action_values = model(state_batch, model.init_hidden()).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    next_state_values = Variable(torch.zeros(BATCH_SIZE).type(Tensor))
    next_state_values[non_final_mask] = model(non_final_next_states).max(1)[0]
    # Now, we don't want to mess up the loss with a volatile flag, so let's
    # clear it. After this, we'll just end up with a Variable that has
    # requires_grad=False
    next_state_values.volatile = False
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values)

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in model.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [9]:
num_episodes = 10
for i_episode in range(num_episodes):
    # Initialize the environment and state
    last_point = get_next_data()
    current_point = get_next_data()
    state = current_point - last_point
    for t in count():
        # Select and perform an action
        action = select_action(state)
        reward, done = sim.step(action[0, 0])
        reward = Tensor([reward])

        # Observe new state
        last_point = current_point
        current_point = get_next_data()
        if not done:
            next_state = current_point - last_point
        else:
            next_state = None
        print(f'working${state}')

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the target network)
        optimize_model()
        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break

print('Complete')


working$

Columns 0 to 5 
 1.1700e+00 -2.9000e-01  5.0000e-05  2.3960e+01  2.8400e+00 -2.6000e-01

Columns 6 to 11 
-1.0000e-05 -4.6400e+00  6.3000e-01 -1.4000e-01  3.0000e-05 -1.2000e+00

Columns 12 to 17 
 7.7000e-01  6.1000e-01 -3.0000e-05  1.4180e+01  7.3023e+02 -2.9943e+02

Columns 18 to 23 
-2.7848e+02  2.1902e+05  5.5779e+05 -1.1867e+05 -7.3400e+00  2.1170e+01

Columns 24 to 29 
 1.1700e+00 -2.9000e-01  5.0000e-05  2.1000e+01  1.0300e+00  1.6000e-01

Columns 30 to 33 
-3.0000e-05  1.0000e+00  0.0000e+00  0.0000e+00
[torch.FloatTensor of size 1x34]

working$

Columns 0 to 5 
 1.2000e+00 -3.6000e-01 -1.0000e-05  3.9000e-01  1.9000e-01 -2.2000e-01

Columns 6 to 11 
 3.0000e-05  2.6110e+01  1.0700e+00  5.2000e-01  0.0000e+00  2.1170e+01

Columns 12 to 17 
 1.1700e+00 -2.9000e-01  0.0000e+00  1.7970e+01  6.6410e+01  1.1844e+02

Columns 18 to 23 
 2.5262e+02  2.7941e+05  5.0806e+04  4.6956e+04  6.6600e+00 -8.8500e+00

Columns 24 to 29 
 1.2000e+00 -3.6000e-01  4.0000e-05 -4.7200e+00  

RuntimeError: inconsistent tensor sizes at /Users/soumith/code/builder/wheel/pytorch-src/torch/lib/TH/generic/THTensorMath.c:2864